<a href="https://colab.research.google.com/github/ElnazDi/colab/blob/master/Three_Approach_Missing_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
upload = files.upload()

Saving melb_data.csv to melb_data.csv


In [8]:
import pandas as pd
mel_data = pd.read_csv('melb_data.csv')
mel_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [9]:
mel_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

# How To Drop a Single Column from a Dataframe?

To drop a single column from pandas dataframe, we need to provide the name of the column to be dropped as a list as an argument to drop function. 

To specify we want to drop column, we need to provide axis=1 as another argument to drop function.

# How to select only numerical values in our data set
we need to exclude it from objects !

In [0]:
y = mel_data.Price                              #just like before we keep price as a actual value in y
                                                #now we want to work at the same data_set so we need to drop this y or price column from our data set
melb_X = mel_data.drop(['Price'], axis=1)       #axis=1 I think it want to make columns fit becaus of this dropping // axis=0 works for rows
X = melb_X.select_dtypes(exclude=['object'])    #we remove all data that are not in numerical format

In [0]:
# Just like before we divide our data to (test and training)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y, train_size=0.8, test_size=0.2, random_state=0)

In [0]:
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_model(X_train, X_val, y_train, y_val):
  my_model= RandomForestRegressor(n_estimators=10, random_state=0)  # Parameters : n_estimators : The number of trees in the forest.
  my_model.fit(X_train, y_train)
  prediction = my_model.predict(X_val)
  mae = mean_absolute_error(y_val, prediction)
  return mae
  

# Score from Approach 1 (Drop Columns with Missing Values)¶

Definition and Usage of:  **any()**

The any() function returns True if any item in an iterable are true, otherwise it returns False.
If the iterable object is empty, the any() function will return False.

Definition and Usage of:  **isnull()**

Column is checked for NULL values and a boolean series is returned by the isnull() method which stores True for ever NaN value and False for a Not null value.

In [16]:
# Get names of columns with missing values

columns_with_missvalues = [col for col in X_train.columns if X_train[col].isnull().any() ]  
#this is enough for both X_train and X_val since they come from common dataset with common columns


# Drop columns in training and validation data
doped_non_cols_in_X_train = X_train.drop(columns_with_missvalues, axis=1)
doped_non_cols_in_X_val = X_val.drop(columns_with_missvalues, axis=1)

print("MAE from approach one: (Drop columns with missing values):")
print(score_model(doped_non_cols_in_X_train, doped_non_cols_in_X_val, y_train, y_val))


MAE from approach one: (Drop columns with missing values):
183550.22137772635


# Score from Approach 2 (Imputation)

Next, we use **SimpleImputer** to replace missing values with the **mean value** along each column.


**fit_transform():**

Jjoins these two stepsfit() and transform() and is used for the initial fitting of parameters on the training set x, but it also returns a transformed x′. Internally, it just calls first fit() and then transform() on the same data.

[Image Explainition](https://pbpython.com/pandas_transform.html)

![alt text](https://pbpython.com/images/transform-example.png)


In [17]:
from sklearn.impute import SimpleImputer

# Imputation

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_val = pd.DataFrame(my_imputer.transform(X_val))

print("MAE from approch two: (Imputation)")
print(score_model(imputed_X_train ,imputed_X_val , y_train, y_val))




MAE from approch two: (Imputation)
178166.46269899711


# Score from Approach 3 (An Extension to Imputation)

In [20]:
# Make copy to avoid changing original data (when imputing)

X_train_plus = X_train.copy()
X_val_plus = X_val.copy()

# Make new columns indicating what will be imputed

for col in columns_with_missvalues:
  X_train_plus[col + '_was missing'] = X_train_plus[col].isnull()
  X_val_plus[col + '_wasmissing'] = X_val_plus[col].isnull()
  
# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_val = pd.DataFrame(my_imputer.transform(X_val_plus))


print("MAE from third approach: (An Extension to Imputation)")
print(score_model(imputed_X_train ,imputed_X_val, y_train, y_val))


MAE from third approach: (An Extension to Imputation)
178927.503183954


In [24]:
print(X_train.shape) #The training data has 10864 rows and 12 columns
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column)

(10864, 12)
Rooms               0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                49
Landsize            0
BuildingArea     5156
YearBuilt        4307
Lattitude           0
Longtitude          0
Propertycount       0
dtype: int64


In [25]:
print(missing_val_count_by_column[missing_val_count_by_column > 0])


Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64
